In [1]:
import pandas as pd
import numpy as np

import os
from __future__ import print_function
import codecs
import pickle
import json

import numpy as np
import pandas as pd
import random

from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from sklearn.manifold import MDS

from scipy.cluster.hierarchy import ward, dendrogram

import nltk
from nltk.stem.snowball import SnowballStemmer
import re
import mpld3

import matplotlib.pyplot as plt

In [2]:
pwd

'/Users/steventa/Desktop/TripAdvisor/OpinRankDatasetWithJudgments/hotels'

In [3]:
rating_headers = ['doc_id','hotel_name','hotel_url','street','city','state','country','zip','class','price','num_reviews','CLEANLINESS','ROOM','SERVICE','LOCATION','VALUE', 'COMFORT','overall_ratingsource']

In [4]:
df_hotels_las_vegas = pd.read_csv('data/las-vegas.csv', names = rating_headers).drop(0)

In [7]:
df_hotels_las_vegas['overall_ratingsource'] = df_hotels_las_vegas['overall_ratingsource'].astype(float)

In [8]:
df_hotels_las_vegas['overall_ratingsource'] > 4

1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21      True
22      True
23     False
24      True
25      True
26      True
27      True
28      True
29      True
30      True
       ...  
201    False
202    False
203    False
204    False
205    False
206    False
207    False
208    False
209    False
210    False
211    False
212    False
213    False
214    False
215    False
216    False
217    False
218    False
219    False
220    False
221    False
222    False
223    False
224    False
225    False
226    False
227    False
228    False
229    False
230    False
Name: overall_ratingsource, Length: 230, dtype: bool

In [9]:
df_hotels_las_vegas.sort_values(['overall_ratingsource', 'num_reviews'], axis=0, ascending=False).head()

,doc_id,hotel_name,hotel_url,street,city,state,country,zip,class,price,num_reviews,CLEANLINESS,ROOM,SERVICE,LOCATION,VALUE,COMFORT,overall_ratingsource
32,usa_nevada_las-vegas_the_mansion_at_mgm_grand,the mansion at mgm grand,http://www.tripadvisor.com/ShowUserReviews-g45...,-1,las vegas,nv,usa,89109,-1,-1,10,5.0,5.0,5.0,5.0,5.0,0.0,5.000000
139,usa_nevada_las-vegas_streamline_tower,streamline tower,http://www.tripadvisor.com/ShowUserReviews-g45...,-1,las vegas,nv,usa,89101,-1,-1,1,5.0,5.0,5.0,5.0,5.0,0.0,5.000000
165,usa_nevada_las-vegas_las_vegas_mini_castle,las vegas mini castle,http://www.tripadvisor.com/ShowUserReviews-g45...,-1,las vegas,nv,usa,89123,-1,-1,1,5.0,5.0,5.0,5.0,5.0,0.0,5.000000
1,usa_nevada_las-vegas_staybridge_suites_las_vegas,staybridge suites las vegas,http://www.tripadvisor.com/ShowUserReviews-g45...,-1,las vegas,nv,usa,89118,-1,-1,113,4.884615384615385,4.913461538461538,4.721153846153846,4.259615384615385,4.855769230769231,0.0,4.726923
5,usa_nevada_las-vegas_skylofts_at_mgm_grand,skylofts at mgm grand,http://www.tripadvisor.com/ShowUserReviews-g45...,-1,las vegas,nv,usa,89109,-1,-1,77,4.863636363636363,4.893939393939394,4.803030303030303,4.53030303030303,4.378787878787879,0.0,4.693939


In [89]:
df_hotels_las_vegas['zip'].value_counts()

89109         75
89101         24
89119         21
89103         13
89104         12
89118         11
89121          7
89123          7
89115          7
89122          6
89169          6
-1             4
89130          3
89135          3
89147          3
89102          2
89107          2
89145          2
89030          2
89117          2
89108          1
89139          1
89052          1
nv 89139       1
89123-1607     1
89019          1
89128-0455     1
89119-1206     1
89124          1
89104-1312     1
89112          1
89109-3611     1
89128          1
89027          1
89183          1
89109-4300     1
89118-1655     1
89101-1011     1
Name: zip, dtype: int64

In [10]:
df_hotels_las_vegas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 1 to 230
Data columns (total 18 columns):
doc_id                  230 non-null object
hotel_name              230 non-null object
hotel_url               230 non-null object
street                  230 non-null object
city                    230 non-null object
state                   230 non-null object
country                 230 non-null object
zip                     230 non-null object
class                   230 non-null object
price                   230 non-null object
num_reviews             230 non-null object
CLEANLINESS             230 non-null object
ROOM                    230 non-null object
SERVICE                 230 non-null object
LOCATION                230 non-null object
VALUE                   230 non-null object
COMFORT                 230 non-null object
overall_ratingsource    230 non-null float64
dtypes: float64(1), object(17)
memory usage: 44.1+ KB


In [11]:
# convert overall_ratingsource to float
df_hotels_san_francisco['overall_ratingsource'] = df_hotels_san_francisco['overall_ratingsource'].astype(float)

NameError: name 'df_hotels_san_francisco' is not defined

In [12]:
df_hotels_san_francisco.info()

NameError: name 'df_hotels_san_francisco' is not defined

In [15]:
review_headers = ['date', 'review_title', 'full_review', 'nan']

In [16]:
review = pd.read_csv('data/las-vegas/usa_nevada_las-vegas_skylofts_at_mgm_grand', 
                     names=review_headers ,sep='\t', encoding='latin-1').drop('nan', axis=1)

In [67]:
review.head()

,date,review_title,full_review
0,NaN,wow,"This hotel is totally awesome. Limo, champagne..."
1,Nov 15 2009,good value,Reasonably priced (for Las Vegas) and great lo...
2,Nov 7 2009,Possibly My Favourite Hotel Experience Ever,"Having stayed in the Burj Al Arab, Dubai and t..."
3,Oct 16 2009,Didn't want to leave,Let me preface this review by saying we had be...
4,Oct 13 2009,Skylofts 1bdrm and Bellagio 1 Bdrm Penthouse,I did two nights at skylofts and two at Bellag...


In [18]:
lst_files = os.listdir('data/las-vegas/')

In [19]:
lst_files.index('usa_nevada_las-vegas_skylofts_at_mgm_grand')

70

In [20]:
for comment in review.iloc[:3,:]['full_review']:
    print(comment)
    print()

This hotel is totally awesome. Limo, champagne, red wine and franch cheese, personnal concierge, hammam and the BED!... so much, you wanna live there!

Reasonably priced (for Las Vegas) and great location. Right by the monorail stop, so its easy to get around to some of the other hotels. We liked it better than another we stayed at in Las Vegas. Staff was very helpful and we were able to buy tickets to a show right at the hotel. We picked a show that was right in the hotel (Ka). The show was great and it was relaxing not to have to go out anywhere to get to the show.

Having stayed in the Burj Al Arab, Dubai and the Conrad, Tokyo I still think this is my favourite hotel in the world. Unobtrusive but attentive staff, amazing decor and a real feeling of been treated as a VIP from the second you arrive. We stayed for a week and everythign was perfect. I recommend this hotel to anyone, provided your pockets are deep. Go for an afternoon to Wet Republic and eat at Joel Robuchon.



In [21]:
df_hotels_las_vegas['overall_ratingsource'] = df_hotels_las_vegas['overall_ratingsource'].astype(float)

In [22]:
df_hotels_las_vegas['num_reviews'] = df_hotels_las_vegas['num_reviews'].astype(int)

In [23]:
not_neg_one = df_hotels_las_vegas['overall_ratingsource'] != -1

In [24]:
# find lowest rating
without_neg_one_df = df_hotels_las_vegas[not_neg_one]

In [25]:
# more than 3 reviews
more_than_three = without_neg_one_df['num_reviews'] > 3
without_neg_one_df = without_neg_one_df[more_than_three]

In [26]:
without_neg_one_df['overall_ratingsource'].min()

1.4666666666666668

In [27]:
# now get min
rating_min = without_neg_one_df['overall_ratingsource'] == without_neg_one_df['overall_ratingsource'].min()

In [28]:
df_min_rating = without_neg_one_df[rating_min]

In [29]:
df_min_rating.head()

,doc_id,hotel_name,hotel_url,street,city,state,country,zip,class,price,num_reviews,CLEANLINESS,ROOM,SERVICE,LOCATION,VALUE,COMFORT,overall_ratingsource
222,usa_nevada_las-vegas_motel_6_las_vegas_boulder...,motel 6 las vegas boulder highway,http://www.tripadvisor.com/ShowUserReviews-g45...,-1,las vegas,nv,usa,89121,-1,-1,6,1.0,1.0,1.6666666666666667,2.6666666666666665,1.0,0.0,1.466667


In [30]:
# look at usa_nevada_las-vegas_motel_6_las_vegas_boulder

In [31]:
review_boulder = pd.read_csv('data/las-vegas/usa_nevada_las-vegas_motel_6_las_vegas_boulder_highway', 
                     names=review_headers ,sep='\t', encoding='latin-1').drop('nan', axis=1)

In [32]:
review_boulder.head()

,date,review_title,full_review
0,Nov 5 2008,Drug needles under the bed!!!!!!!!,This motel is absolutely terrible. There was a...
1,Mar 26 2007,This place stinks - literally!!!,"We requested a non-smoking room. What a joke, ..."
2,Feb 7 2007,Rather Sleep In A Cardboard Box,My wife and I were going to stay here for the ...
3,Aug 20 2006,Yuck! Double Yuck!,If you like feeling like you're sleeping in an...
4,Jun 1 2006,The WORST place you may ever stay.,"The Motel 6 on Boulder Hwy. in Las vegas, NV i..."


In [33]:
# try to find experience of this review

one_review = '''
We stayed here for 8 nights on our trip to San Francisco from Australia, and we absolutely loved it. 
From our arrival and check-in, to check-out, service was faultless and facilities were wonderful. 
We got a 2 bedroom apartment, with 2 bathrooms, and a fully equipped kitchen. 
All appliances were state of the art, and our room also had a Bosch washer and a separate dryer, 
which was really helpful. The apartment itself was very spacious, with large windows, automatic blinds, 
3 large high-def LCD screens, cable TV with 100's of channels, DVD players - as well as a gas heater in the 
living area! We loved it so much we wished that we lived at Fairmong Heritage Place Ghirardelli Square. 
Also downstairs, there are 3 or 4 Ghirardelli chocolate cafe's and shops, which were fantastic. 
There are several restaurants as well as specialty shops at the Square, and it's only a short walk from the 
Fisherman's Wharf itself. Staff were fantastic, with the concierge doing everything he could to accommodate 
to our requests, from in-room dining, to arranging a in-residence masseuse for my wife. The Maserati town car 
was absolutely fantastic as well, as it took us to Union Square, shopping areas, or other restaurants that are 
a just a bit too far to walk to. There's a guest lounge area as well, with a high quality pool table and 
daily breakfast in the morning. We loved it, and so will you.
'''

In [34]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [35]:
# load nltk's SnowballStemmer as variabled 'stemmer'
stemmer = SnowballStemmer("english")

In [36]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [37]:
hotel_names, hotel_reviews = 'usa_nevada_las-vegas_skylofts_at_mgm_grand', one_review

In [38]:
hotel_vocab_stemmed = []
hotel_vocab_tokenized = []

allwords_stemmed = tokenize_and_stem(one_review)
hotel_vocab_stemmed.extend(allwords_stemmed)
allwords_tokenized = tokenize_only(one_review)
hotel_vocab_tokenized.extend(allwords_tokenized)

In [39]:
len(hotel_vocab_stemmed), len(hotel_vocab_tokenized)

(240, 240)

In [40]:
hotel_vocab_frame = pd.DataFrame({'words': hotel_vocab_tokenized}, index = hotel_vocab_stemmed)

In [41]:
hotel_vocab_frame

,words
we,we
stay,stayed
here,here
for,for
night,nights
on,on
our,our
trip,trip
to,to
san,san


In [42]:
pwd

'/Users/steventa/Desktop/TripAdvisor/OpinRankDatasetWithJudgments/hotels'

In [43]:
cd ~/Desktop

/Users/steventa/Desktop


In [44]:
cd TripAdvisorSentiment/

/Users/steventa/Desktop/TripAdvisorSentiment


In [53]:
import json
import pandas as pd
data = json.load(open('100407.json'))

test = pd.DataFrame(data["Reviews"])

In [63]:
test.head()

,Author,AuthorLocation,Content,Date,Ratings,ReviewID,Title
0,Nadine R,"Jacksonville, Florida","Usually stay near the airport, but this trip w...","April 19, 2012","{'Service': '5', 'Cleanliness': '5', 'Overall'...",UR128178025,“Great place and location”
1,blt3116,"Chillicothe, Ohio",Stayed at this Hilton for 2 nights. It was lik...,"January 30, 2012","{'Service': '4', 'Cleanliness': '4', 'Overall'...",UR123847354,"“Nice stay, nice surroundings”"
2,ExSpec5,Northern Virginia,"Stayed there one night, December 16, on the wa...","December 28, 2011","{'Service': '5', 'Cleanliness': '5', 'Overall'...",UR122192448,“Perfect for an overnight just off I-95”
3,kevinlynch3,Raleigh,I just stayed here last weekend and have alrea...,"December 23, 2011","{'Service': '5', 'Cleanliness': '5', 'Overall'...",UR122031890,“Great Hotel”
4,nobumponalog,"Belmont, MA",My mother who is 90 and I stayed one night on ...,"October 14, 2011","{'Service': '5', 'Cleanliness': '5', 'Overall'...",UR119295575,“good room for handicapped person”


In [65]:
test.Ratings.iloc[2]

{'Cleanliness': '5',
 'Location': '5',
 'Overall': '4.0',
 'Rooms': '5',
 'Service': '5',
 'Sleep Quality': '3',
 'Value': '3'}

In [72]:
file_list = os.listdir()

In [73]:
len(file_list)

258

In [71]:
cd TripAdvisor/OpinRankDatasetWithJudgments/hotels/data/new-york-city/

/Users/steventa/Desktop/TripAdvisor/OpinRankDatasetWithJudgments/hotels/data/new-york-city


In [77]:
review_headers = ['date', 'review_title', 'full_review', 'nan']

In [78]:
def get_count(file_name):

        ''' get all hotel reviews for city '''

        df_reviews = pd.read_csv(file_name,
        names=review_headers, sep='\t', encoding='latin-1').drop('nan', axis=1)

        return df_reviews.shape[0]

In [81]:
count = 0

for idx, name in enumerate(file_list):
    if idx == 31 or idx == 45:
        continue
    else:
        count += get_count(name)

In [82]:
count

54513

In [83]:
file_list[0]

'usa_new york city_the_st_regis_hotel_new_york'

In [84]:
test_df = pd.read_csv('usa_new york city_the_st_regis_hotel_new_york',
        names=review_headers, sep='\t', encoding='latin-1').drop('nan', axis=1)

In [87]:
test_df.shape[0]

121

In [88]:
file_list[:100]

['usa_new york city_the_st_regis_hotel_new_york',
 'usa_new york city_the_muse_hotel_new_york_a_kimpton_hotel',
 'usa_new york city_holiday_inn_nyc_manhattan_6th_avenue',
 'usa_new york city_new_york_palace_hotel',
 'usa_new york city_the_bowery_hotel',
 'usa_new york city_the_sherry_netherland_hotel',
 'usa_new york city_70_park_avenue_hotel_a_kimpton_hotel',
 'usa_new york city_park_79_hotel',
 'usa_new york city_the_pierre',
 'usa_new york city_hotel_mela',
 'usa_new york city_club_quarters_downtown',
 'usa_new york city_best_western_bowery_hanbee_hotel',
 'usa_new york city_phillips_club',
 'usa_new york city_the_hampton_inn_times_square_north',
 'usa_new york city_amsterdam_court_hotel',
 'usa_new york city_the_benjamin',
 'usa_new york city_the_time',
 'usa_new york city_thirty_thirty_hotel',
 'usa_new york city_hotel_metro',
 'usa_new york city_hotel_elysee',
 'usa_new york city_sofitel_new_york',
 'usa_new york city_doubletree_metropolitan_hotel',
 'usa_new york city_murray_hil